<a href="https://colab.research.google.com/github/mezhcoder/onti_ai_2020/blob/master/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Установка библиотек**

In [0]:
#!pip install catboost
#!wget -O data_v1.zip —no-check-certificate "https://onti-2019.s3-eu-central-1.amazonaws.com/public/data_v1.zip"          
#!unzip data_v1.zip  

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import catboost as cb
import time
import os
from sklearn.model_selection import cross_val_score

plt.style.use('ggplot')

**Чтение файлов**


In [0]:
train_data = pd.read_csv('data/transactions_train.csv')
train_target = pd.read_csv('data/train_target.csv')

test_data = pd.read_csv('data/transactions_test.csv')
test_target = pd.read_csv('data/test.csv')

**Функции для манипулирования с обучающими данными**

In [0]:
def group_by(data, groupList, value, typeArg, name=''):
    if typeArg == 'count':
        new_data = data.groupby(groupList)[value].count().reset_index()
    elif typeArg == 'sum':
        new_data = data.groupby(groupList)[value].sum().reset_index()
    elif typeArg == 'mean':
        new_data = data.groupby(groupList)[value].mean().reset_index()
    elif typeArg == 'std':
        new_data = data.groupby(groupList)[value].std().reset_index()  
    elif typeArg == 'min':
        new_data = data.groupby(groupList)[value].min().reset_index()  
    elif typeArg == 'max':
        new_data = data.groupby(groupList)[value].max().reset_index()        
    categories = new_data.reset_index().pivot(index=groupList[0], columns=groupList[1], values=value)
    categories = categories.fillna(0)
    categories.columns = [name + str(i) for i in categories.columns]
    return categories  

In [0]:
def merge_all_df(dataframes, p):
    df = dataframes[0]
    for i in range(1, len(dataframes)):
        df = df.merge(dataframes[i], on=p)
    return df

**Манипулирование с обучающими данными**

In [0]:
train_table_agg_features = train_data.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()
train_table_agg_features.head()

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


In [0]:
#train_table_categories_count - показывает сколько соверишилось количество покупок в определенной категории
#train_table_categories_sum - показывает сумму купленных покупок в определенной категории
#итак по аналогии..
train_table_categories_count = group_by(train_data, ['client_id','small_group'], 'amount_rur', 'count', name='sg_count_')
train_table_categories_sum = group_by(train_data, ['client_id','small_group'], 'amount_rur', 'sum', name='sg_sum_')
train_table_categories_std = group_by(train_data, ['client_id','small_group'], 'amount_rur', 'std', name='sg_std_')
train_table_categories_min = group_by(train_data, ['client_id','small_group'], 'amount_rur', 'min', name='sg_min_')
train_table_categories_max = group_by(train_data, ['client_id','small_group'], 'amount_rur', 'max', name='sg_max_')
train_table_categories_mean = group_by(train_data, ['client_id','small_group'], 'amount_rur', 'mean', name='sg_mean_')

In [0]:
train_data['weekday'] = train_data.trans_date % 7
train_table_days_sum = group_by(train_data, ['client_id', 'weekday'], 'amount_rur', 'sum', 'wd_sum_')
train_table_days_count = group_by(train_data, ['client_id', 'weekday'], 'amount_rur', 'count', 'wd_count_')
train_table_days_std = group_by(train_data, ['client_id', 'weekday'], 'amount_rur', 'std', 'wd_std_')
train_table_days_min = group_by(train_data, ['client_id', 'weekday'], 'amount_rur', 'min', 'wd_min_')
train_table_days_max = group_by(train_data, ['client_id', 'weekday'], 'amount_rur', 'max', 'wd_max_')
train_table_days_mean = group_by(train_data, ['client_id', 'weekday'], 'amount_rur', 'mean', 'wd_mean_')

In [0]:
train_data['month'] = train_data.trans_date//30.4167 % 12
train_table_month_sum = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'sum', 'month_sum_')
train_table_month_count = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'count', 'month_count_')
train_table_month_std = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'std', 'month_std_')
train_table_month_min = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'min', 'month_min_')
train_table_month_max = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'max', 'month_max_')
train_table_month_mean = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'mean', 'month_mean_')

In [0]:
train_table_month_sum = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'sum', 'month_sum_')
train_table_month_count = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'count', 'month_count_')
train_table_month_std = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'std', 'month_std_')
train_table_month_min = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'min', 'month_min_')
train_table_month_max = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'max', 'month_max_')
train_table_month_mean = group_by(train_data, ['client_id', 'month'], 'amount_rur', 'mean', 'month_mean_')

In [0]:
train_first_temp_table = None
train_first_temp_table = train_table_categories_sum
d = [train_first_temp_table,train_table_categories_count,train_table_categories_std,train_table_categories_min,train_table_categories_max,train_table_categories_mean,
     train_table_days_sum,train_table_days_count,train_table_days_std,train_table_days_min,train_table_days_max,train_table_days_mean,
     train_table_month_sum,train_table_month_count,train_table_month_std,train_table_month_min,train_table_month_max,train_table_month_mean]
train_result_table = merge_all_df(d, 'client_id')

In [0]:
train_data_driver = train_data[(train_data['small_group'] == 55) | (train_data['small_group'] == 76) | (train_data['small_group'] == 121) | (train_data['small_group'] == 138)]

train_table_driver_sum = group_by(train_data_driver, ['client_id', 'small_group'], 'amount_rur', 'sum', 'driver_sum_')
train_table_driver_sum['driver_sum'] = train_table_driver_sum.sum(axis=1)

train_table_driver_count = group_by(train_data_driver, ['client_id', 'small_group'], 'amount_rur', 'count', 'driver_count_')
train_table_driver_count['driver_count'] = train_table_driver_count.count(axis=1)

train_table_driver_std = group_by(train_data_driver, ['client_id', 'small_group'], 'amount_rur', 'std', 'driver_std_')
train_table_driver_std['driver_std'] = train_table_driver_std.std(axis=1)

train_table_driver_min = group_by(train_data_driver, ['client_id', 'small_group'], 'amount_rur', 'min', 'driver_min_')
train_table_driver_min['driver_min'] = train_table_driver_min.min(axis=1)

train_table_driver_max = group_by(train_data_driver, ['client_id', 'small_group'], 'amount_rur', 'max', 'driver_max_')
train_table_driver_max['driver_max'] = train_table_driver_max.max(axis=1)

train_table_driver_mean = group_by(train_data_driver, ['client_id', 'small_group'], 'amount_rur', 'mean', 'driver_mean_')
train_table_driver_mean['driver_mean'] = train_table_driver_mean.mean(axis=1)

d = [train_table_driver_sum['driver_sum'].to_frame(), train_table_driver_count['driver_count'].to_frame(),
     train_table_driver_std['driver_std'].to_frame(), train_table_driver_min['driver_min'].to_frame(), 
     train_table_driver_max['driver_max'].to_frame(),
     train_table_driver_mean['driver_mean'].to_frame()
     ]
train_table_driver_result = merge_all_df(d, 'client_id')

train_result_table = pd.merge(train_result_table, train_table_driver_result, on='client_id', how='left')
train_result_table = train_result_table.fillna(0)

In [0]:
train_data_driver = train_data[(train_data['small_group'] == 55) | (train_data['small_group'] == 76) | (train_data['small_group'] == 121) | (train_data['small_group'] == 138)]

train_wd_table_driver_sum = group_by(train_data_driver, ['client_id', 'weekday'], 'amount_rur', 'sum', 'driver_sum_')
train_wd_table_driver_sum['wd_driver_sum'] = train_wd_table_driver_sum.sum(axis=1)

train_wd_table_driver_count = group_by(train_data_driver, ['client_id', 'weekday'], 'amount_rur', 'count', 'driver_count_')
train_wd_table_driver_count['wd_driver_count'] = train_wd_table_driver_count.count(axis=1)

train_wd_table_driver_std = group_by(train_data_driver, ['client_id', 'weekday'], 'amount_rur', 'std', 'driver_std_')
train_wd_table_driver_std['wd_driver_std'] = train_wd_table_driver_std.std(axis=1)

train_wd_table_driver_min = group_by(train_data_driver, ['client_id', 'weekday'], 'amount_rur', 'min', 'driver_min_')
train_wd_table_driver_min['wd_driver_min'] = train_wd_table_driver_min.min(axis=1)

train_wd_table_driver_max = group_by(train_data_driver, ['client_id', 'weekday'], 'amount_rur', 'max', 'driver_max_')
train_wd_table_driver_max['wd_driver_max'] = train_wd_table_driver_max.max(axis=1)

train_wd_table_driver_mean = group_by(train_data_driver, ['client_id', 'weekday'], 'amount_rur', 'mean', 'driver_mean_')
train_wd_table_driver_mean['wd_driver_mean'] = train_wd_table_driver_mean.mean(axis=1)

d = [train_wd_table_driver_sum['wd_driver_sum'].to_frame(), train_wd_table_driver_count['wd_driver_count'].to_frame(),
     train_wd_table_driver_std['wd_driver_std'].to_frame(), train_wd_table_driver_min['wd_driver_min'].to_frame(), 
     train_wd_table_driver_max['wd_driver_max'].to_frame(),
     train_wd_table_driver_mean['wd_driver_mean'].to_frame()
     ]
train_table_driver_result = merge_all_df(d, 'client_id')

train_result_table = pd.merge(train_result_table, train_table_driver_result, on='client_id', how='left')
train_result_table = train_result_table.fillna(0)

In [0]:
train_data_young = train_data[(train_data['small_group'] == 67) | (train_data['small_group'] == 54) | (train_data['small_group'] == 79) | (train_data['small_group'] == 104)]

train_table_young_sum = group_by(train_data_young, ['client_id', 'small_group'], 'amount_rur', 'sum', 'young_sum_')
train_table_young_sum['young_sum'] = train_table_young_sum.sum(axis=1)

train_table_young_count = group_by(train_data_young, ['client_id', 'small_group'], 'amount_rur', 'count', 'young_count_')
train_table_young_count['young_count'] = train_table_young_count.count(axis=1)

train_table_young_std = group_by(train_data_young, ['client_id', 'small_group'], 'amount_rur', 'std', 'young_std_')
train_table_young_std['young_std'] = train_table_young_std.std(axis=1)

train_table_young_min = group_by(train_data_young, ['client_id', 'small_group'], 'amount_rur', 'min', 'young_min_')
train_table_young_min['young_min'] = train_table_young_min.min(axis=1)

train_table_young_max = group_by(train_data_young, ['client_id', 'small_group'], 'amount_rur', 'max', 'young_max_')
train_table_young_max['young_max'] = train_table_young_max.max(axis=1)

train_table_young_mean = group_by(train_data_young, ['client_id', 'small_group'], 'amount_rur', 'mean', 'young_mean_')
train_table_young_mean['young_mean'] = train_table_young_mean.mean(axis=1)

d = [train_table_young_sum['young_sum'].to_frame(), train_table_young_count['young_count'].to_frame(),
     train_table_young_std['young_std'].to_frame(), train_table_young_min['young_min'].to_frame(), 
     train_table_young_max['young_max'].to_frame(),
     train_table_young_mean['young_mean'].to_frame()
     ]
train_table_young_result = merge_all_df(d, 'client_id')

train_result_table = pd.merge(train_result_table, train_table_young_result, on='client_id', how='left')
train_result_table = train_result_table.fillna(0)

In [0]:
train_data_young = train_data[(train_data['small_group'] == 67) | (train_data['small_group'] == 54) | (train_data['small_group'] == 79) | (train_data['small_group'] == 104)]

train_wd_table_young_sum = group_by(train_data_young, ['client_id', 'weekday'], 'amount_rur', 'sum', 'young_sum_')
train_wd_table_young_sum['wd_young_sum'] = train_wd_table_young_sum.sum(axis=1)

train_wd_table_young_count = group_by(train_data_young, ['client_id', 'weekday'], 'amount_rur', 'count', 'young_count_')
train_wd_table_young_count['wd_young_count'] = train_wd_table_young_count.count(axis=1)

train_wd_table_young_std = group_by(train_data_young, ['client_id', 'weekday'], 'amount_rur', 'std', 'young_std_')
train_wd_table_young_std['wd_young_std'] = train_wd_table_young_std.std(axis=1)

train_wd_table_young_min = group_by(train_data_young, ['client_id', 'weekday'], 'amount_rur', 'min', 'young_min_')
train_wd_table_young_min['wd_young_min'] = train_wd_table_young_min.min(axis=1)

train_wd_table_young_max = group_by(train_data_young, ['client_id', 'weekday'], 'amount_rur', 'max', 'young_max_')
train_wd_table_young_max['wd_young_max'] = train_wd_table_young_max.max(axis=1)

train_wd_table_young_mean = group_by(train_data_young, ['client_id', 'weekday'], 'amount_rur', 'mean', 'young_mean_')
train_wd_table_young_mean['wd_young_mean'] = train_wd_table_young_mean.mean(axis=1)

d = [train_wd_table_young_sum['wd_young_sum'].to_frame(), train_wd_table_young_count['wd_young_count'].to_frame(),
     train_wd_table_young_std['wd_young_std'].to_frame(), train_wd_table_young_min['wd_young_min'].to_frame(), 
     train_wd_table_young_max['wd_young_max'].to_frame(),
     train_wd_table_young_mean['wd_young_mean'].to_frame()
     ]
train_wd_table_young_result = merge_all_df(d, 'client_id')

train_result_table = pd.merge(train_result_table, train_wd_table_young_result, on='client_id', how='left')
train_result_table = train_result_table.fillna(0)

In [0]:
train_result_table

,sg_sum_0,sg_sum_1,sg_sum_2,sg_sum_3,sg_sum_4,sg_sum_5,sg_sum_6,sg_sum_7,sg_sum_8,sg_sum_9,sg_sum_10,sg_sum_11,sg_sum_12,sg_sum_13,sg_sum_14,sg_sum_15,sg_sum_16,sg_sum_17,sg_sum_18,sg_sum_19,sg_sum_20,sg_sum_21,sg_sum_22,sg_sum_23,sg_sum_24,sg_sum_25,sg_sum_26,sg_sum_27,sg_sum_28,sg_sum_29,sg_sum_30,sg_sum_31,sg_sum_32,sg_sum_33,sg_sum_34,sg_sum_35,sg_sum_36,sg_sum_37,sg_sum_38,sg_sum_39,...,month_max_8,month_max_9,month_max_10,month_max_11,month_mean_0,month_mean_1,month_mean_2,month_mean_3,month_mean_4,month_mean_5,month_mean_6,month_mean_7,month_mean_8,month_mean_9,month_mean_10,month_mean_11,driver_sum,driver_count,driver_std,driver_min,driver_max,driver_mean,wd_driver_sum,wd_driver_count,wd_driver_std,wd_driver_min,wd_driver_max,wd_driver_mean,young_sum,young_count,young_std,young_min,young_max,young_mean,wd_young_sum,wd_young_count,wd_young_std,wd_young_min,wd_young_max,wd_young_mean
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.000,15049.897,98.966,835.403,3799.572,0.000,0.0,0.000,22.852,514.861,4.167,192.051,0.000,0.000,0.0,776.921,0.000,0.0,138.800,0.000,725.786,0.000,0.000,282.421,422.416,0.000,0.000,0.0,12.895,0.0,0.000,0.000,56.128,0.000,125.111,0.000,244.622,0.000,0.000,1457.402,...,198.387,295.624,249.840,1341.802,32.555179,40.789345,40.273955,32.320459,36.015123,33.328850,43.502661,50.900364,29.471764,35.687806,34.056255,61.018889,0.000,0.0,0.000000,0.0,0.000,0.00000,0.000,0.0,0.00000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
6,53.372,9602.770,0.000,2905.083,189.480,0.000,0.0,0.000,0.000,80.193,147.555,241.624,0.000,0.000,0.0,982.398,0.000,0.0,14.425,0.000,205.328,0.000,0.000,0.000,0.000,0.000,0.000,0.0,304.050,0.0,13.671,0.000,0.000,0.000,22.285,0.000,0.000,30.456,0.000,0.000,...,298.702,86.255,43.057,85.181,25.344884,27.489942,17.597808,20.123850,19.565241,23.799360,18.374432,23.871636,27.490782,21.314397,10.581676,21.727562,0.000,0.0,0.000000,0.0,0.000,0.00000,0.000,0.0,0.00000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
7,89.072,4632.372,125.521,888.523,660.002,1241.557,0.0,118.975,197.820,3568.778,0.000,7004.386,3720.717,5423.866,0.0,152.876,993.303,0.0,6180.351,260.232,0.000,301.422,104.070,575.033,1677.721,665.408,0.000,0.0,9448.947,0.0,0.000,26.739,76.936,64.238,128.827,91.145,1676.861,452.572,0.000,232.673,...,603.786,238.360,1422.780,4505.971,41.611236,78.529935,53.641717,146.404014,49.195014,82.479632,48.594641,40.303600,56.277079,44.775904,84.375393,97.963917,0.000,0.0,0.000000,0.0,0.000,0.00000,0.000,0.0,0.00000,0.0,0.000,0.000000,720.806,4.0,29.522840,0.0,188.899,53.467786,720.806,7.0,27.345298,0.0,188.899,56.956143
10,639.798,13440.506,24.928,2034.764,2422.679,0.000,0.0,0.000,270.330,722.931,0.000,2314.969,0.000,0.000,0.0,1394.209,0.000,0.0,141.598,69.696,0.000,0.000,0.000,553.715,3103.754,606.832,0.000,0.0,1251.864,0.0,3.435,131.539,0.000,0.000,259.958,0.000,1292.937,0.000,0.000,840.036,...,504.378,205.805,147.141,654.893,58.426203,46.193898,58.422827,41.464790,46.976180,64.330977,39.261577,33.765500,53.085045,43.013672,40.864588,57.077368,0.000,0.0,0.000000,0.0,0.000,0.00000,0.000,0.0,0.00000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
11,88.239,5037.288,0.000,637.526,178.423,1844.468,0.0,0.000,0.000,86.990,192.291,1705.113,0.000,38.815,0.0,46.953,2.463,0.0,748.705,0.000,31.420,108.077,2100.073,306.454,1004.501,1929.789,393.245,0.0,0.000,0.0,0.000,0.000,115.151,0.000,62.535,247.190,820.429,0.000,0.000,83.667,...,685.729,184.786,223.832,352.165,38.391056,33.757778,26.419182,77.914784,26.598605,24.766567,50.736014,25.359248,42.682200,13.677717,23.526103,26.647028,0.000,0.0,0.000000,0.0,0.000,0.00000,0.000,0.0,0.00000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

**Манипулирование с тестовыми данными**

In [0]:
test_table_agg_features = test_data.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()
test_table_agg_features.head()

,client_id,sum,mean,std,min,max
0,0,17036.127,19.163247,40.561700,0.065,595.339
1,1,34748.964,47.666617,89.489278,0.298,1181.221
2,2,51188.069,68.433247,152.093601,0.043,2837.682
3,3,47975.203,67.857430,318.651653,0.043,6135.652
4,5,20059.100,21.803370,34.258433,0.043,439.902


In [0]:
test_table_categories_count = group_by(test_data, ['client_id','small_group'], 'amount_rur', 'count', name='sg_count_')
test_table_categories_sum = group_by(test_data, ['client_id','small_group'], 'amount_rur', 'sum', name='sg_sum_')
test_table_categories_std = group_by(test_data, ['client_id','small_group'], 'amount_rur', 'std', name='sg_std_')
test_table_categories_min = group_by(test_data, ['client_id','small_group'], 'amount_rur', 'min', name='sg_min_')
test_table_categories_max = group_by(test_data, ['client_id','small_group'], 'amount_rur', 'max', name='sg_max_')
test_table_categories_mean = group_by(test_data, ['client_id','small_group'], 'amount_rur', 'mean', name='sg_mean_')

In [0]:
test_data['weekday'] = test_data.trans_date % 7
test_table_days_sum = group_by(test_data, ['client_id', 'weekday'], 'amount_rur', 'sum', 'wd_sum_')
test_table_days_count = group_by(test_data, ['client_id', 'weekday'], 'amount_rur', 'count', 'wd_count_')
test_table_days_std = group_by(test_data, ['client_id', 'weekday'], 'amount_rur', 'std', 'wd_std_')
test_table_days_min = group_by(test_data, ['client_id', 'weekday'], 'amount_rur', 'min', 'wd_min_')
test_table_days_max = group_by(test_data, ['client_id', 'weekday'], 'amount_rur', 'max', 'wd_max_')
test_table_days_mean = group_by(test_data, ['client_id', 'weekday'], 'amount_rur', 'mean', 'wd_mean_')

In [0]:
test_data['month'] = test_data.trans_date//30.4167 % 12
test_table_month_sum = group_by(test_data, ['client_id', 'month'], 'amount_rur', 'sum', 'month_sum_')
test_table_month_count = group_by(test_data, ['client_id', 'month'], 'amount_rur', 'count', 'month_count_')
test_table_month_std = group_by(test_data, ['client_id', 'month'], 'amount_rur', 'std', 'month_std_')
test_table_month_min = group_by(test_data, ['client_id', 'month'], 'amount_rur', 'min', 'month_min_')
test_table_month_max = group_by(test_data, ['client_id', 'month'], 'amount_rur', 'max', 'month_max_')
test_table_month_mean = group_by(test_data, ['client_id', 'month'], 'amount_rur', 'mean', 'month_mean_')

In [0]:
test_first_temp_table = None
test_first_temp_table = test_table_categories_sum
d = [test_first_temp_table,test_table_categories_count,test_table_categories_std,test_table_categories_min,test_table_categories_max,test_table_categories_mean,
     test_table_days_sum,test_table_days_count,test_table_days_std,test_table_days_min,test_table_days_max,test_table_days_mean,
     test_table_month_sum,test_table_month_count,test_table_month_std,test_table_month_min,test_table_month_max,test_table_month_mean]
test_result_table = merge_all_df(d, 'client_id')

In [0]:
test_data_driver = test_data[(test_data['small_group'] == 55) | (test_data['small_group'] == 76) | (test_data['small_group'] == 121) | (test_data['small_group'] == 138)]

test_table_driver_sum = group_by(test_data_driver, ['client_id', 'small_group'], 'amount_rur', 'sum', 'driver_sum_')
test_table_driver_sum['driver_sum'] = test_table_driver_sum.sum(axis=1)

test_table_driver_count = group_by(test_data_driver, ['client_id', 'small_group'], 'amount_rur', 'count', 'driver_count_')
test_table_driver_count['driver_count'] = test_table_driver_count.count(axis=1)

test_table_driver_std = group_by(test_data_driver, ['client_id', 'small_group'], 'amount_rur', 'std', 'driver_std_')
test_table_driver_std['driver_std'] = test_table_driver_std.std(axis=1)

test_table_driver_min = group_by(test_data_driver, ['client_id', 'small_group'], 'amount_rur', 'min', 'driver_min_')
test_table_driver_min['driver_min'] = test_table_driver_min.min(axis=1)

test_table_driver_max = group_by(test_data_driver, ['client_id', 'small_group'], 'amount_rur', 'max', 'driver_max_')
test_table_driver_max['driver_max'] = test_table_driver_max.max(axis=1)

test_table_driver_mean = group_by(test_data_driver, ['client_id', 'small_group'], 'amount_rur', 'mean', 'driver_mean_')
test_table_driver_mean['driver_mean'] = test_table_driver_mean.mean(axis=1)

d = [test_table_driver_sum['driver_sum'].to_frame(), test_table_driver_count['driver_count'].to_frame(),
     test_table_driver_std['driver_std'].to_frame(), test_table_driver_min['driver_min'].to_frame(), 
     test_table_driver_max['driver_max'].to_frame(),
     test_table_driver_mean['driver_mean'].to_frame()
     ]
test_table_driver_result = merge_all_df(d, 'client_id')

test_result_table = pd.merge(test_result_table, test_table_driver_result, on='client_id', how='left')
test_result_table = test_result_table.fillna(0)

In [0]:
test_data_driver = test_data[(test_data['small_group'] == 55) | (test_data['small_group'] == 76) | (test_data['small_group'] == 121) | (test_data['small_group'] == 138)]

test_wd_table_driver_sum = group_by(test_data_driver, ['client_id', 'weekday'], 'amount_rur', 'sum', 'driver_sum_')
test_wd_table_driver_sum['wd_driver_sum'] = test_wd_table_driver_sum.sum(axis=1)

test_wd_table_driver_count = group_by(test_data_driver, ['client_id', 'weekday'], 'amount_rur', 'count', 'driver_count_')
test_wd_table_driver_count['wd_driver_count'] = test_wd_table_driver_count.count(axis=1)

test_wd_table_driver_std = group_by(test_data_driver, ['client_id', 'weekday'], 'amount_rur', 'std', 'driver_std_')
test_wd_table_driver_std['wd_driver_std'] = test_wd_table_driver_std.std(axis=1)

test_wd_table_driver_min = group_by(test_data_driver, ['client_id', 'weekday'], 'amount_rur', 'min', 'driver_min_')
test_wd_table_driver_min['wd_driver_min'] = test_wd_table_driver_min.min(axis=1)

test_wd_table_driver_max = group_by(test_data_driver, ['client_id', 'weekday'], 'amount_rur', 'max', 'driver_max_')
test_wd_table_driver_max['wd_driver_max'] = test_wd_table_driver_max.max(axis=1)

test_wd_table_driver_mean = group_by(test_data_driver, ['client_id', 'weekday'], 'amount_rur', 'mean', 'driver_mean_')
test_wd_table_driver_mean['wd_driver_mean'] = test_wd_table_driver_mean.mean(axis=1)

d = [test_wd_table_driver_sum['wd_driver_sum'].to_frame(), test_wd_table_driver_count['wd_driver_count'].to_frame(),
     test_wd_table_driver_std['wd_driver_std'].to_frame(), test_wd_table_driver_min['wd_driver_min'].to_frame(), 
     test_wd_table_driver_max['wd_driver_max'].to_frame(),
     test_wd_table_driver_mean['wd_driver_mean'].to_frame()
     ]
test_table_driver_result = merge_all_df(d, 'client_id')

test_result_table = pd.merge(test_result_table, test_table_driver_result, on='client_id', how='left')
test_result_table = test_result_table.fillna(0)

In [0]:
test_data_young = test_data[(test_data['small_group'] == 67) | (test_data['small_group'] == 54) | (test_data['small_group'] == 79) | (test_data['small_group'] == 104)]

test_table_young_sum = group_by(test_data_young, ['client_id', 'small_group'], 'amount_rur', 'sum', 'young_sum_')
test_table_young_sum['young_sum'] = test_table_young_sum.sum(axis=1)

test_table_young_count = group_by(test_data_young, ['client_id', 'small_group'], 'amount_rur', 'count', 'young_count_')
test_table_young_count['young_count'] = test_table_young_count.count(axis=1)

test_table_young_std = group_by(test_data_young, ['client_id', 'small_group'], 'amount_rur', 'std', 'young_std_')
test_table_young_std['young_std'] = test_table_young_std.std(axis=1)

test_table_young_min = group_by(test_data_young, ['client_id', 'small_group'], 'amount_rur', 'min', 'young_min_')
test_table_young_min['young_min'] = test_table_young_min.min(axis=1)

test_table_young_max = group_by(test_data_young, ['client_id', 'small_group'], 'amount_rur', 'max', 'young_max_')
test_table_young_max['young_max'] = test_table_young_max.max(axis=1)

test_table_young_mean = group_by(test_data_young, ['client_id', 'small_group'], 'amount_rur', 'mean', 'young_mean_')
test_table_young_mean['young_mean'] = test_table_young_mean.mean(axis=1)

d = [test_table_young_sum['young_sum'].to_frame(), test_table_young_count['young_count'].to_frame(),
     test_table_young_std['young_std'].to_frame(), test_table_young_min['young_min'].to_frame(), 
     test_table_young_max['young_max'].to_frame(),
     test_table_young_mean['young_mean'].to_frame()
     ]
test_table_young_result = merge_all_df(d, 'client_id')

test_result_table = pd.merge(test_result_table, test_table_young_result, on='client_id', how='left')
test_result_table = test_result_table.fillna(0)

In [0]:
test_data_young = test_data[(test_data['small_group'] == 67) | (test_data['small_group'] == 54) | (test_data['small_group'] == 79) | (test_data['small_group'] == 104)]

test_wd_table_young_sum = group_by(test_data_young, ['client_id', 'weekday'], 'amount_rur', 'sum', 'young_sum_')
test_wd_table_young_sum['wd_young_sum'] = test_wd_table_young_sum.sum(axis=1)

test_wd_table_young_count = group_by(test_data_young, ['client_id', 'weekday'], 'amount_rur', 'count', 'young_count_')
test_wd_table_young_count['wd_young_count'] = test_wd_table_young_count.count(axis=1)

test_wd_table_young_std = group_by(test_data_young, ['client_id', 'weekday'], 'amount_rur', 'std', 'young_std_')
test_wd_table_young_std['wd_young_std'] = test_wd_table_young_std.std(axis=1)

test_wd_table_young_min = group_by(test_data_young, ['client_id', 'weekday'], 'amount_rur', 'min', 'young_min_')
test_wd_table_young_min['wd_young_min'] = test_wd_table_young_min.min(axis=1)

test_wd_table_young_max = group_by(test_data_young, ['client_id', 'weekday'], 'amount_rur', 'max', 'young_max_')
test_wd_table_young_max['wd_young_max'] = test_wd_table_young_max.max(axis=1)

test_wd_table_young_mean = group_by(test_data_young, ['client_id', 'weekday'], 'amount_rur', 'mean', 'young_mean_')
test_wd_table_young_mean['wd_young_mean'] = test_wd_table_young_mean.mean(axis=1)

d = [test_wd_table_young_sum['wd_young_sum'].to_frame(), test_wd_table_young_count['wd_young_count'].to_frame(),
     test_wd_table_young_std['wd_young_std'].to_frame(), test_wd_table_young_min['wd_young_min'].to_frame(), 
     test_wd_table_young_max['wd_young_max'].to_frame(),
     test_wd_table_young_mean['wd_young_mean'].to_frame()
     ]
test_wd_table_young_result = merge_all_df(d, 'client_id')

test_result_table = pd.merge(test_result_table, test_wd_table_young_result, on='client_id', how='left')
test_result_table = test_result_table.fillna(0)

In [0]:
test_result_table

,sg_sum_0,sg_sum_1,sg_sum_2,sg_sum_3,sg_sum_4,sg_sum_5,sg_sum_6,sg_sum_7,sg_sum_8,sg_sum_9,sg_sum_10,sg_sum_11,sg_sum_12,sg_sum_13,sg_sum_14,sg_sum_15,sg_sum_16,sg_sum_17,sg_sum_18,sg_sum_19,sg_sum_20,sg_sum_21,sg_sum_22,sg_sum_23,sg_sum_24,sg_sum_25,sg_sum_26,sg_sum_27,sg_sum_28,sg_sum_29,sg_sum_30,sg_sum_31,sg_sum_32,sg_sum_33,sg_sum_34,sg_sum_35,sg_sum_36,sg_sum_37,sg_sum_38,sg_sum_39,...,month_max_8,month_max_9,month_max_10,month_max_11,month_mean_0,month_mean_1,month_mean_2,month_mean_3,month_mean_4,month_mean_5,month_mean_6,month_mean_7,month_mean_8,month_mean_9,month_mean_10,month_mean_11,driver_sum,driver_count,driver_std,driver_min,driver_max,driver_mean,wd_driver_sum,wd_driver_count,wd_driver_std,wd_driver_min,wd_driver_max,wd_driver_mean,young_sum,young_count,young_std,young_min,young_max,young_mean,wd_young_sum,wd_young_count,wd_young_std,wd_young_min,wd_young_max,wd_young_mean
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.000,4994.452,18.456,425.071,159.784,0.000,0.000,0.000,325.916,517.858,0.000,2528.354,470.364,0.000,0.000,92.547,414.966,3.675,1541.779,0.000,0.000,0.000,0.000,111.355,640.107,735.494,61.198,0.0,600.035,11.712,45.795,62.967,128.227,24.653,0.000,130.651,753.954,0.000,0.000,54.908,...,109.733,206.150,175.559,318.368,11.386920,19.047596,10.752621,17.712554,27.152051,15.387840,12.981468,27.065451,13.411109,18.682021,20.366846,35.183016,115.797,4.0,8.920152,0.0,68.357,15.013750,115.797,7.0,10.373478,0.0,68.357,13.153857,16.028,4.0,0.0,0.0,9.159,4.00700,16.028,7.0,0.0,0.0,9.159,2.289714
1,1467.718,9847.510,0.000,780.896,1781.393,0.000,0.000,0.000,0.000,11941.055,0.000,40.109,0.000,0.000,0.000,1604.604,802.537,0.000,0.000,0.000,513.021,42.050,22.898,15.051,3821.617,624.070,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,298.353,123.843,0.000,12.982,...,324.354,740.889,330.047,403.823,82.103097,105.235760,74.372083,54.719800,28.989964,31.479382,32.987463,28.094747,33.820154,55.036392,31.705326,46.904644,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.0,0.0,0.000,0.00000,0.000,0.0,0.0,0.0,0.000,0.000000
2,983.132,9984.946,30.189,2367.933,4318.762,2570.389,0.000,169.861,60.816,2012.776,464.164,880.809,693.910,2552.398,0.000,581.563,275.762,1170.899,2040.735,0.000,1278.121,323.456,154.846,2270.678,3431.909,911.524,312.951,0.0,1556.521,58.739,6.412,287.013,1223.270,64.691,1350.150,79.698,1560.562,0.000,0.000,0.000,...,122.225,559.527,802.104,606.917,65.827914,80.286045,51.354794,47.521387,92.999137,49.341643,95.828453,49.896093,33.481000,90.102849,82.612619,86.895412,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,1051.002,4.0,0.0,0.0,1051.002,262.75050,1051.002,7.0,0.0,0.0,1051.002,150.143143
3,0.000,9790.742,6099.548,778.271,741.149,6078.311,0.000,452.604,210.052,1807.067,0.000,3330.951,0.000,0.000,330.736,805.633,460.788,348.501,276.501,0.000,239.445,520.780,61.933,468.303,1728.954,322.404,2.588,0.0,288.454,0.000,0.000,207.936,0.000,30.189,501.888,0.000,0.000,115.264,0.000,0.000,...,256.317,850.433,437.357,229.297,47.128841,55.735311,263.808825,42.603885,36.890164,42.430881,55.327245,78.145492,51.705477,47.252600,49.339118,50.127164,1427.977,4.0,95.352624,0.0,744.173,30.505518,1427.977,7.0,153.789889,0.0,744.173,101.746024,22.898,4.0,0.0,0.0,22.898,5.72450,22.898,7.0,0.0,0.0,22.898,3.271143
5,89.816,11365.690,7.763,1721.041,449.996,0.000,0.000,0.000,325.408,165.476,25.877,902.473,11.757,0.000,0.000,888.079,305.244,0.000,88.395,16.575,0.000,30.189,144.692,185.528,515.016,520.572,0.000,0.0,0.000,0.000,561.802,0.000,0.000,0.000,3.664,75.363,188.988,0.000,0.000,32.739,...,95.942,113.421,439.902,185.299,22.430976,29.535947,24.186800,21.418342,22.209311,19.755387,16.832600,25.420148,19.243395,18.669926,19.518706,23.875929,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.0,0.0,0.000,0.00000,0.000,0.0,0.0,0.0,0.000,0.000000
...,...,...,...,...,...,...

**Алгоритмизация и создание итоговых таблиц для обучения (train, test)**

In [0]:
train = pd.merge(train_target, train_table_agg_features, on='client_id')
train = pd.merge(train, train_result_table.reset_index(), on='client_id')

In [0]:
train

,client_id,bins,sum,mean,std,min,max,sg_sum_0,sg_sum_1,sg_sum_2,sg_sum_3,sg_sum_4,sg_sum_5,sg_sum_6,sg_sum_7,sg_sum_8,sg_sum_9,sg_sum_10,sg_sum_11,sg_sum_12,sg_sum_13,sg_sum_14,sg_sum_15,sg_sum_16,sg_sum_17,sg_sum_18,sg_sum_19,sg_sum_20,sg_sum_21,sg_sum_22,sg_sum_23,sg_sum_24,sg_sum_25,sg_sum_26,sg_sum_27,sg_sum_28,sg_sum_29,sg_sum_30,sg_sum_31,sg_sum_32,...,month_max_8,month_max_9,month_max_10,month_max_11,month_mean_0,month_mean_1,month_mean_2,month_mean_3,month_mean_4,month_mean_5,month_mean_6,month_mean_7,month_mean_8,month_mean_9,month_mean_10,month_mean_11,driver_sum,driver_count,driver_std,driver_min,driver_max,driver_mean,wd_driver_sum,wd_driver_count,wd_driver_std,wd_driver_min,wd_driver_max,wd_driver_mean,young_sum,young_count,young_std,young_min,young_max,young_mean,wd_young_sum,wd_young_count,wd_young_std,wd_young_min,wd_young_max,wd_young_mean
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,0.000,7093.826,64.110,1412.978,729.946,0.000,0.000,0.000,13.281,177.969,0.000,2962.656,8176.869,0.000,0.000,166.896,0.000,0.000,1173.039,0.000,0.000,575.347,202.034,90.579,1828.179,156.983,64.691,0.0,308.918,0.00,0.000,91.440,0.000,...,84.584,228.976,262.338,258.766,28.393571,37.201952,28.808513,53.564778,29.312258,30.675769,26.938218,44.896382,27.745264,27.082727,40.574490,44.726784,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,1227.314,4.0,0.000000,0.0,1227.314,306.828500,1227.314,7.0,0.000000,0.0,1227.314,175.330571
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,95.346,10920.764,3712.276,607.346,454.812,605.997,0.000,0.000,185.060,427.985,71.212,754.638,58.021,0.000,5.175,1199.835,0.000,0.000,4449.666,0.000,131.342,0.000,335.340,27.727,2199.132,30.014,398.029,0.0,2498.317,10.89,0.000,223.045,0.000,...,1144.424,272.651,577.020,430.338,40.715604,69.419629,62.436471,62.575471,64.978737,45.320907,40.561013,54.384985,62.336268,37.249455,37.251704,43.846218,1547.076,4.0,282.302570,0.0,1210.506,96.692250,1547.076,7.0,0.000000,0.0,1210.506,221.010857,1163.199,4.0,3.238196,0.0,586.179,145.399875,1163.199,7.0,0.000000,0.0,586.179,166.171286
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,0.000,9969.533,0.000,795.089,440.769,1509.061,0.000,0.000,0.000,1900.341,0.000,675.530,269.849,0.000,0.000,548.327,1305.423,8.022,285.318,0.000,185.103,185.448,19.234,1549.467,2771.367,385.290,0.000,0.0,439.111,0.00,0.000,0.000,0.000,...,328.353,274.680,320.109,782.641,28.916909,50.053549,24.689066,19.726809,25.398295,32.572389,31.985395,35.287346,39.042233,30.254691,44.012507,45.028867,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,159.102,4.0,10.266596,0.0,61.549,23.516667,159.102,7.0,1.762321,0.0,61.549,14.407143
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,0.000,7985.775,0.733,0.000,248.769,0.000,0.000,0.000,0.000,373.859,0.000,1137.966,0.000,0.000,0.000,4667.600,0.000,0.000,492.946,0.000,21.995,0.000,0.000,0.000,0.000,65.892,0.000,0.0,29.858,0.00,0.000,21.478,0.000,...,64.591,54.409,74.574,73.772,18.391857,15.386447,12.258771,14.746143,17.649923,20.818732,15.606851,16.937700,15.547726,11.920396,18.553575,17.304266,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,183.560,4.0,6.267142,0.0,52.831,4.171818,183.560,7.0,5.799586,0.0,52.831,9.465592
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,0.000,3907.408,0.000,3821.081,554.640,0.000,0.000,0.000,0.000,1015.106,0.000,463.211,0.000,123.647,0.000,3.889,0.000,0.000,3.450,0.000,0.000,0.000,791.305,116.089,683.760,404.388,0.000,0.0,155.259,0.00,0.000,60.379,0.000,...,107.985,332.082,68.693,128.003,11.342870,10.928634,19.275708,11.346469,26.093788,10.299544,13.494225,19.167800,14.642250,35.335290,11.937697,17.664261,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
test = pd.merge(test_target, test_table_agg_features, on='client_id')
test = pd.merge(test, test_result_table.reset_index(), on='client_id')

In [0]:
test

,client_id,sum,mean,std,min,max,sg_sum_0,sg_sum_1,sg_sum_2,sg_sum_3,sg_sum_4,sg_sum_5,sg_sum_6,sg_sum_7,sg_sum_8,sg_sum_9,sg_sum_10,sg_sum_11,sg_sum_12,sg_sum_13,sg_sum_14,sg_sum_15,sg_sum_16,sg_sum_17,sg_sum_18,sg_sum_19,sg_sum_20,sg_sum_21,sg_sum_22,sg_sum_23,sg_sum_24,sg_sum_25,sg_sum_26,sg_sum_27,sg_sum_28,sg_sum_29,sg_sum_30,sg_sum_31,sg_sum_32,sg_sum_33,...,month_max_8,month_max_9,month_max_10,month_max_11,month_mean_0,month_mean_1,month_mean_2,month_mean_3,month_mean_4,month_mean_5,month_mean_6,month_mean_7,month_mean_8,month_mean_9,month_mean_10,month_mean_11,driver_sum,driver_count,driver_std,driver_min,driver_max,driver_mean,wd_driver_sum,wd_driver_count,wd_driver_std,wd_driver_min,wd_driver_max,wd_driver_mean,young_sum,young_count,young_std,young_min,young_max,young_mean,wd_young_sum,wd_young_count,wd_young_std,wd_young_min,wd_young_max,wd_young_mean
0,28571,30507.083,42.488974,43.659666,0.078,306.882,0.000,13241.983,327.129,692.735,1062.961,0.000,0.000,0.000,12.852,576.089,109.847,911.056,13.372,0.000,0.000,517.909,591.439,0.000,566.471,0.000,0.000,137.386,0.000,1414.032,3488.858,1042.000,124.554,0.0,47.347,0.000,0.000,178.740,22.858,621.115,...,202.384,228.976,189.775,173.347,47.625977,46.265265,51.238278,36.997963,34.384281,41.804015,37.432918,44.786088,37.972776,48.504500,41.771552,42.196208,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
1,27046,39378.016,38.988135,93.600961,0.043,1469.007,468.444,4156.274,2848.273,717.378,561.533,0.000,283.830,0.000,25.004,3938.199,205.162,4597.807,467.993,1591.873,0.000,718.749,2823.708,0.000,2114.925,36.658,0.000,852.169,0.000,668.332,4712.650,253.778,135.660,0.0,31.965,0.000,21.294,217.862,19.903,40.579,...,841.191,197.956,551.284,1360.013,61.205750,71.791687,40.536919,30.434357,29.789524,30.591104,37.568750,25.982754,55.295783,24.327913,33.670778,48.352738,428.738,4.0,36.641685,0.0,184.929,27.974425,428.738,7.0,38.998166,0.0,184.929,30.716571,127.391,4.0,3.263290,0.0,51.322,22.736750,127.391,7.0,8.943096,0.0,51.322,13.257143
2,13240,50211.127,53.302683,198.583630,1.078,3902.918,0.000,12482.813,34.575,1635.920,1400.729,0.000,0.000,319.679,208.461,2101.954,154.619,3865.498,0.000,0.000,0.000,410.185,1203.500,0.000,692.661,0.000,33.946,6051.377,8.701,752.142,4331.871,498.802,0.000,0.0,850.235,0.000,0.000,43.559,66.065,0.000,...,353.387,3902.918,514.601,2019.571,86.855378,37.638526,35.283645,42.940125,35.897793,47.984171,73.886582,38.338068,33.388146,81.283624,38.816648,75.889448,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000,1586.389,4.0,0.000000,0.0,1497.088,396.597250,1586.389,7.0,0.000000,0.0,1497.088,226.627000
3,19974,45371.491,53.252924,273.597147,0.432,5865.551,1417.209,6965.268,861.671,967.675,1395.934,0.000,0.000,0.000,157.120,823.387,54.125,2067.716,62.145,287.799,0.000,29.685,90.864,0.000,101.426,36.636,0.000,1060.401,137.340,1099.891,9348.230,1775.763,0.000,0.0,667.617,11.564,158.493,77.112,2341.138,16.029,...,5151.511,504.378,603.356,5865.551,54.241556,40.246944,46.102667,32.070463,32.542260,36.881898,32.426620,46.715938,112.625941,51.813371,34.002092,109.892989,6.869,4.0,0.000000,0.0,6.869,1.717250,6.869,7.0,0.000000,0.0,6.869,0.981286,0.000,0.0,0.000000,0.0,0.000,0.000000,0.000,0.0,0.000000,0.0,0.000,0.000000
4,10505,90883.628,81.950972,119.408720,0.043,1921.341,0.000,54857.382,2700.061,2063.610,2709.704,1921.341,0.000,966.141,422.949,2754.977,69.392,1110.732,0.000,0.000,1883.997,886.327,12.019,0.000,870.818,33.384,0.000,298.877,405.723,904.084,5564.403,0.000,455.931,0.0,795.132,126.545,6.090,191.730,10.782,0.000,...,966.141,775.865,412.493,512.790,65.818354,106.233171,77.499690,71.213530,93.275620,68.042344,82.423962,83.506478,82.943948,96.243570,76.783376,80.156317,185.158,4.0,10.966033,0.0,70.525,11.572375,185.158,7.0,0.000000,0.0,70.525,26.451143,19.408,4.0,0.000000,0.0,19.408,4.852000,19.408,7.0,0.000000,0.0,19.408,2.772571
..

**Обучение**

In [0]:
common_features=list(set(train.columns).intersection(set(test.columns)))
common_features.pop(common_features.index('client_id'))
len(common_features)

1343

In [0]:
y_train=train['bins']
X_train=train[common_features]
X_test=test[common_features]

In [0]:
param={'objective':'multi:softprob', 'num_class':4, 'n_jobs':4, 'seed':42}

In [0]:
%%time
model = cb.CatBoostClassifier(iterations=24300, task_type='GPU')
model.fit(X_train, y_train)

In [0]:
#array([0, 2, 3, ..., 0, 2, 3])
pred = model.predict(X_test)
pred = np.reshape(pred, len(pred)).astype(int)
pred

array([0, 2, 3, ..., 2, 2, 3])

**Проверка результатов**

In [0]:
%%time
scores = cross_val_score(model, X_train, y_train, cv=5)

In [0]:
scores

In [0]:
np.mean(scores)

**Создание результатов и их загрузка**

In [0]:
submission = pd.DataFrame({'bins': pred}, index=test.client_id)
submission.head()

,bins
client_id,
28571,0
27046,2
13240,3
19974,2
10505,1


In [0]:
current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)
submission.to_csv(submission_path, index=True)

In [0]:
time.time()